In [ ]:
# kaggle cats n dogs dataset
# https://www.kaggle.com/chetankv/dogs-cats-images?
!wget -O 'dataset' 'https://storage.googleapis.com/kaggle-data-sets/22535%2F28903%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1597926548&Signature=HOJaYq0f8svILz%2FF5zbi8cLTP%2F7PfIVMnHm65mfFK2KKBSrCndPmh3vbx5%2BuoAanCNNzDELDcsi%2BUlZLgu0Ao110LRZiosDCoNbYDfCBXx7pcadSAYaEuL6SSuQV2HzkGOMjesuFYvUO99S7aI1XBgfw3R4zvQfRULRC4dyRBbfNIh871cnqLdQ%2FLrp%2Bp%2FJn%2BTm8Toas7sqzKuXxgpp%2BIqkPe7MLcnff6Jz%2FAO0sbMgSRysv0BjwhoGG5eKSTh8%2BD2tRhCBOJeEVlnwxzW8NtuKNR%2FwNF%2BThh56SRbLMZxmF0hCBEm6ME5BuDxYqy1c3A4ZXFheKPdYLbnAPvQz6hw%3D%3D'

--2020-08-17 12:29:59--  https://storage.googleapis.com/kaggle-data-sets/22535%2F28903%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1597926548&Signature=HOJaYq0f8svILz%2FF5zbi8cLTP%2F7PfIVMnHm65mfFK2KKBSrCndPmh3vbx5%2BuoAanCNNzDELDcsi%2BUlZLgu0Ao110LRZiosDCoNbYDfCBXx7pcadSAYaEuL6SSuQV2HzkGOMjesuFYvUO99S7aI1XBgfw3R4zvQfRULRC4dyRBbfNIh871cnqLdQ%2FLrp%2Bp%2FJn%2BTm8Toas7sqzKuXxgpp%2BIqkPe7MLcnff6Jz%2FAO0sbMgSRysv0BjwhoGG5eKSTh8%2BD2tRhCBOJeEVlnwxzW8NtuKNR%2FwNF%2BThh56SRbLMZxmF0hCBEm6ME5BuDxYqy1c3A4ZXFheKPdYLbnAPvQz6hw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455718914 (435M) [application/zip]
Saving to: ‘dataset’

dataset             100%[===================>] 434.61M  35.5MB/s    in 15s     

2020-08-17 12:30

In [ ]:
src_zip = 'dataset'
cpy_zip = 'dataset_folder/' 

import zipfile
with zipfile.ZipFile(src_zip, 'r') as zip_ref:
    zip_ref.extractall(cpy_zip)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import random

from keras.utils.np_utils import to_categorical


In [ ]:
def make_lab_dict(path_dataset):
    folders = os.listdir(path_dataset)
    folders = np.sort(folders)  
    labs = list(range(0,len(folders)))
    lab_dict = dict(zip(folders, labs))
    
    return lab_dict

def get_paths(path_dataset):
    all_paths = glob(path_dataset+'/'+'*'+'/'+'*')
    return all_paths

In [ ]:
def my_datagen(path_dataset, lab_dict, batch_size=64, out_shape=(128,128,3)):
    
    while True: 
        all_paths = get_paths(path_dataset)
        selected_paths = random.sample(all_paths, batch_size)
        
        batch_images = []
        batch_labels = []

        for i in range(0,len(selected_paths)):
            img = cv2.imread(selected_paths[i])
            img = cv2.resize(img,(out_shape[0],out_shape[1]))

            img_category = selected_paths[i].split('/')[-2]
            lab = lab_dict[img_category]
            lab = to_categorical(lab, num_classes=2)
            # print(img_category)
            # print(lab)

            batch_images.append(img)
            batch_labels.append(lab)
        
        yield np.array(batch_images)/255, np.array(batch_labels)

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow as tf

In [ ]:
def build_model(input_shape = (128,128,3)):
        
    k_size = 3
    filters = 32
    strides = 2
    pad = 'same'
    acti = 'relu'
    

    inp_model = Input(shape=(input_shape))
    l1 = Conv2D(filters=filters, kernel_size=k_size, strides=strides, padding=pad, activation=acti)(inp_model)
    l2 = Conv2D(filters=filters*2, kernel_size=k_size, strides=strides, padding=pad, activation=acti)(l1)
    l3 = Conv2D(filters=filters*4, kernel_size=k_size, strides=strides, padding=pad, activation=acti)(l2)
    l4 = Conv2D(filters=filters*4, kernel_size=k_size, strides=strides, padding=pad, activation=acti)(l3)

    f1 = Flatten()(l4)
    c1 = Dense(256, activation= acti)(f1)
    c2 = Dense(2, activation = 'softmax')(c1)
    model = Model(inp_model, c2)

    return model

In [ ]:
def init_model(model_name='my_model', path_ckpt='checkpoint/'):
    complete_path = path_ckpt+model_name+'.h5'

    model = build_model()
       
    if not os.path.isdir(path_ckpt):
        os.mkdir(path_ckpt)

    names = os.listdir(path_ckpt)
    if complete_path.split('/')[-1] in names:
        print('Loading checkpoint...')
        model.load_weights(complete_path)
    else:
        print('Training from scratch')

    return model, complete_path

    

In [ ]:
def train_model(path_dataset, batch_size=64, epochs=2):
    model, checkpoint_path = init_model()
    
    checkpoint=ModelCheckpoint(filepath=checkpoint_path,monitor='loss',save_best_only=True,
                           save_weights_only=False,verbose=0,mode="auto")
    
    # checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', save_best_only=False, 
    #                              save_weights_only=False, mode='auto', period=2)

    callbacks=[checkpoint]
    print('learning rate = ',learn_rate)
    opt = tf.keras.optimizers.Adam(lr = learn_rate)
    
    model.compile(optimizer=opt,loss="categorical_crossentropy")

    lab_dict = make_lab_dict(path_dataset)
    print(lab_dict)
    print(model.summary())
    data_size = len(get_paths(path_dataset))

    #train_gen = my_datagen(path_dataset, lab_dict, batch_size=batch_size)
    #model.fit_generator(train_gen, epochs=epochs, callbacks = callbacks,steps_per_epoch=int(data_size/batch_size))

    x_train,y_train = next(my_datagen(path_dataset, lab_dict, batch_size=2000))
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, callbacks = callbacks)


    # save trained model
    #model.save('checkpoint/' + 'trained_model.h5')


In [ ]:
global learn_rate
learn_rate = 0.0001

batch_size = 512
input_shape = (128,128,3)
path_dataset = 'dataset_folder/dataset/test_set/'

In [ ]:
##train_model(path_dataset, batch_size, 200)


In [ ]:
###!cp '/content/checkpoint/my_model.h5' '/content/drive/My Drive/Colab Notebooks/change_input_size_of_pretrained_network/'

In [ ]:
def inference(path_dataset, batch_size):
    lab_dict = make_lab_dict(path_dataset)
    x_train,y_train = next(my_datagen(path_dataset, lab_dict, batch_size=batch_size))


    model, checkpoint_path = init_model()
    model.load_weights('/content/drive/My Drive/Colab Notebooks/change_input_size_of_pretrained_network/my_model.h5')

    correct_preds = 0

    preds_all = []
    for i in range(0,batch_size):
        img = x_train[i]
        img = np.expand_dims(img, axis=0)

        lab_org = np.argmax(y_train[i])
        
        pred = model.predict(img).squeeze()
        lab_pred = np.argmax(pred)

        preds_all.append(lab_pred)

        print('---------------------------------------')
        print(i, '-->', 'original label = ',lab_org, '--predicted label = ',lab_pred)
        print('original ', y_train[i],' --prediction = ',pred)

        if lab_org==lab_pred:
            correct_preds = correct_preds+1
        
    print('accuracy = ', (correct_preds*100)/batch_size, ' %')




In [ ]:
##inference(path_dataset,2000)

In [ ]:
def resized_model(input_shape):

    model, checkpoint_path = init_model()
    model.load_weights('/content/drive/My Drive/Colab Notebooks/change_input_size_of_pretrained_network/my_model.h5')


    new_model = build_model(input_shape=input_shape)
    

    for i in range(0,len(model.layers)-3):
        print('-----------------------------')
        print(i)
        print(model.layers[i].name)

        extracted_weights = model.layers[i].get_weights()
        new_model.layers[i].set_weights(extracted_weights)

        print(np.shape(extracted_weights))
        if i !=0:
            print(extracted_weights[0].shape)

    return new_model

In [ ]:
new_mod = resized_model(input_shape=(256,256,3))

Training from scratch
-----------------------------
0
input_7
(0,)
-----------------------------
1
conv2d_24
(2,)
(3, 3, 3, 32)
-----------------------------
2
conv2d_25
(2,)
(3, 3, 32, 64)
-----------------------------
3
conv2d_26
(2,)
(3, 3, 64, 128)
-----------------------------
4
conv2d_27
(2,)
(3, 3, 128, 128)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
mm = VGG16(weights = 'imagenet',input_shape=(224,224,3))

In [ ]:
mm.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_78 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
m2 = VGG16(include_top=False,weights = 'imagenet',input_shape=(400,400,3))

In [ ]:
m2.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_79 (InputLayer)        [(None, 400, 400, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 400, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 400, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 200, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 200, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 200, 200, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 100, 100, 128)     0     